In [39]:
'''
column_type = label
df[column] = df[column].apply(lambda x : parse(x,column_type))


huerustic_predict(column_in) -> label
'''

import re
import pandas as pd
from dateutil.parser import parse

In [32]:

int_pattern = re.compile('\d*')
float_pattern = re.compile('\d*[.]\d*')
date_pattern = '' # HARD AF 

predictions = {'int_test':(True, 'int','Scaled'),
               'date':(True, 'date',None),
               'int_num':(True, 'int','Scaled'),
               'float_num':(True, 'float','Scaled'),
               'int_num_str':(True, 'int','Scaled'),
               'float_num_str':(True, 'float','Scaled')
              }

int_test=['126 mins', '134 mins', '253_mins', '123,000', '53 seconds']
date =['12/08/2012', '12 Aug 2022', '12/08/22','20-08-12','12-08-2021']
int_num =[1,2000,346980,481464,654654]
float_num =[13654.543,3546645.454,54654654.88,64655432.54654,6544453213.654521]
int_num_str =['1','2000','346980','481464654654','64654']
float_num_str =['13654.543','3546645.454','54654654.88','64655432.54654','6544453213.654521']


df = pd.DataFrame({'int_test': int_test,
                    'date': date,
                    'int_num':int_num,
                    'float_num':float_num,
                    'int_num_str': int_num_str,
                    'float_num_str':float_num_str})
#or val in vals:
#   matches = re.findall(int_pattern, val)
#   m = "".join(matches).strip()
#   print(int(m))


'''
CSV_IN
GET_PREDICTIONS (ENSEMBLE OF MODELS THAT VOTE FOR 1 TYPE)
CHOOSE_OPTIONS (TRANSFORMATIONS/ETC.)
PARSE COLUMNS into predicted TYPE
APPLY USER CHOSEN TRANSFORMATIONS/ETC.
'''
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   int_test       5 non-null      object 
 1   date           5 non-null      object 
 2   int_num        5 non-null      int64  
 3   float_num      5 non-null      float64
 4   int_num_str    5 non-null      object 
 5   float_num_str  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [28]:
df_pred = pd.DataFrame(predictions)
df_pred.set_axis(['include','type','transform'], axis=0,inplace=True)
df_pred = df_pred.T
df_pred.reset_index(inplace=True)
df_pred.rename(columns={'index':'column_name'},inplace=True)
df_transf = df_pred.loc[df_pred['include']==True]
df_transf_int = df_transf.loc[df_transf['type']=='int']
df_transf_float = df_transf.loc[df_transf['type']=='float']
df_transf_date = df_transf.loc[df_transf['type']=='date']
df_transf_int

,column_name,include,type,transform
0,int_test,True,int,Scaled
2,int_num,True,int,Scaled
4,int_num_str,True,int,Scaled


In [29]:
for column_name in df:
    if column_name in list(df_transf_int['column_name']):
        int_column = []
        for row in df[column_name].iteritems():
            matches = re.findall(int_pattern, str(row[1]))
            m = "".join(matches).strip()
            int_column.append(int(m))
        df[column_name] = int_column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   int_test       5 non-null      int64  
 1   date           5 non-null      object 
 2   int_num        5 non-null      int64  
 3   float_num      5 non-null      float64
 4   int_num_str    5 non-null      int64  
 5   float_num_str  5 non-null      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 368.0+ bytes


In [42]:
for column_name in df:
    if column_name in list(df_transf_date['column_name']):
        date_column = []
        for row in df[column_name].iteritems():
            str(row[1])
            m = parse(str(row[1]))
            date_column.append(m)
        df[column_name] = date_column
df.info()

[datetime.datetime(2012, 12, 8, 0, 0), datetime.datetime(2022, 8, 12, 0, 0), datetime.datetime(2022, 12, 8, 0, 0), datetime.datetime(2012, 8, 20, 0, 0), datetime.datetime(2021, 12, 8, 0, 0)]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   int_test       5 non-null      object        
 1   date           5 non-null      datetime64[ns]
 2   int_num        5 non-null      int64         
 3   float_num      5 non-null      float64       
 4   int_num_str    5 non-null      object        
 5   float_num_str  5 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 368.0+ bytes
